In [2]:
%load_ext autoreload
%autoreload 2

# install libraries that are not part of Colab by default
!pip install census us mapclassify -q

from maptools import ui, census_vars
from IPython.display import Markdown as md
from census import Census
import us
import pandas as pd
import geopandas as gpd
import requests



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:

# education attainment
url = f"https://api.census.gov/data/2023/acs/acs1/subject?get=group(S1501)&ucgid=pseudo(0100000US$0400000)"
meta_url = "https://api.census.gov/data/2023/acs/acs1/subject"

df, vars = census_vars.get(url, meta_url)
vars


{'ucgid': 'ucgid',
 'S1501_C02_007E': 'Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade',
 'S1501_C02_008E': 'Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma',
 'S1501_C02_005E': "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher",
 'S1501_C02_006E': 'Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over',
 'S1501_C02_003E': 'Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency)',
 'S1501_C02_004E': "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree",
 'S1501_C02_001E': 'Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years',
 'S1501_C02_002E': 'Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school gradu

In [7]:

list(df.columns)

['Geography',
 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years',
 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate',
 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency)',
 "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree",
 "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher",
 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over',
 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade',
 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma',
 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency)',
 'Estimate!!Total!!AGE BY EDUCATI

In [ ]:

meta = requests.get(meta_url).json()
json = requests.get(url).json()
data = pd.DataFrame(json[1:], columns=json[0])
data

In [185]:

# vars = meta["dataset"]["c_variables"]
# vars = requests.get(vars).json()
vars_url = meta["dataset"][0]["c_variablesLink"]
geo_url = meta["dataset"][0]["c_geographyLink"]

# for k, v in meta["dataset"][0].items():
#     print(k, v)

vars = requests.get(vars_url).json()
vars = vars["variables"]
geo = requests.get(geo_url).json()


In [189]:

field_names = dict()
for k, v in vars.items():
    if k in json[0]:
        if "predicateOnly" in v and v["predicateOnly"]:
            field_names[k] = k
        else:
            field_names[k] = v["label"]
# data.rename(columns=field_names)
data.columns

Index(['B01003_001E', 'B01003_001EA', 'B01003_001M', 'B01003_001MA', 'GEO_ID',
       'NAME', 'ucgid'],
      dtype='object')

In [165]:
index_cols = dict()

def indexes(k, v):
    parts = v.split("!!")
    t = tuple(parts[:-1])
    index_cols.setdefault(t, []).append(k)


[indexes(k, v) for k, v in field_names.items()]
longest = max(len(k) for k in index_cols.keys())
keys = list(index_cols.keys())
keys.sort(key=lambda x: (len(x), x))

def make_multi(k, v):
    df = data[v].copy().reset_index(drop=True)

    def f(x):
        n = field_names[x].split("!!")[-1]
        t = k + (n, )
        if len(k) < longest:
            t = t + ("",) * (longest - len(k))
        return t
    
    t = [f(c) for c in v]

    ix = pd.MultiIndex.from_tuples(t)
    df.columns = ix
    return df


indexed_data = [make_multi(k, index_cols[k]) for k in keys]

pd.concat(indexed_data, axis=1)

,ucgid,Geography
0,0100000US,0100000US


In [3]:
url = "http://api.census.gov/data/2023/acs/acs1/subject/variables.json"
resp = requests.get(url)
json = resp.json()

df = pd.DataFrame(json['variables']).T

df["var_name"] = df.label.apply(census_vars.col_name)
col_map = df["var_name"].to_dict()
cols = set(data.columns) & set(col_map.keys())
cols = sorted(list(cols))

df[df.index.isin(cols)].reset_index()["label"].iloc[0]


'Estimate!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:!!Without an Internet subscription'

In [4]:

col_map['[["GEO_ID"'] = 'GEOID'
col_map["NAME"] = 'state'
cols = set(data.columns) & set(col_map.keys())
cols = sorted(list(cols))
table = data[list(cols)].copy()
table = table.rename(columns=col_map)
table.columns


Index(['state', 'households', 'has_one_or_more_types_of_computing_devices',
       'desktop_or_laptop',
       'desktop_or_laptop_with_no_other_type_of_computing_device',
       'smartphone', 'smartphone_with_no_other_type_of_computing_device',
       'tablet_or_other_portable_wireless_computer',
       'tablet_or_other_portable_wireless_computer_with_no_other_type_of_computing_device',
       'other_computer',
       'other_computer_with_no_other_type_of_computing_device', 'no_computer',
       'with_an_internet_subscription',
       'dial_up_with_no_other_type_of_internet_subscription',
       'broadband_of_any_type', 'cellular_data_plan',
       'cellular_data_plan_with_no_other_type_of_internet_subscription',
       'broadband_such_as_cable_fiber_optic_or_dsl',
       'satellite_internet_service', 'without_an_internet_subscription',
       'less_than_20_000', 'with_dial_up_internet_subscription_alone',
       'with_a_broadband_internet_subscription',
       'without_an_internet_sub

In [5]:
url = "https://api.census.gov/data/2023/acs/acs1/subject?get=group(S2801)&ucgid=pseudo(0100000US$0400000)"
data = pd.read_csv(url)
table = census_vars.rename_columns(data)
table

,GEOID,NAME,households,has_one_or_more_types_of_computing_devices,desktop_or_laptop,desktop_or_laptop_with_no_other_type_of_computing_device,smartphone,smartphone_with_no_other_type_of_computing_device,tablet_or_other_portable_wireless_computer,tablet_or_other_portable_wireless_computer_with_no_other_type_of_computing_device,...,without_an_internet_subscription,20_000_to_74_999,with_dial_up_internet_subscription_alone,with_a_broadband_internet_subscription,without_an_internet_subscription,75_000_or_more,with_dial_up_internet_subscription_alone,with_a_broadband_internet_subscription,without_an_internet_subscription,ucgid]
0,"[""0400000US01""",Alabama,2051545,1940724,1479526,44395,1857116,297002,1215121,16485,...,24.9,-888888888,0.1,87.8,12.1,-888888888,0.1,96.4,3.5,0400000US01]
1,"[""0400000US02""",Alaska,276852,269648,229564,5292,260997,26347,178184,1174,...,20.1,-888888888,0.1,89.3,10.6,-888888888,0.1,97.6,2.3,0400000US02]
2,"[""0400000US04""",Arizona,2907014,2818465,2417106,64577,2708607,253699,1911929,14164,...,23.2,-888888888,0.2,90.5,9.2,-888888888,0.1,97.0,3.0,0400000US04]
3,"[""0400000US05""",Arkansas,1232871,1171045,878558,27121,1126328,208396,680511,7305,...,28.8,-888888888,0.1,87.8,12.1,-888888888,0.0,96.0,3.9,0400000US05]
4,"[""0400000US06""",California,13699816,13315679,11497837,244092,12885390,1125514,9255623,71173,...,18.8,-888888888,0.1,91.9,8.0,-888888888,0.1,97.6,2.4,0400000US06]
5,"[""0400000US08""",Colorado,2428261,2371266,2124065,46958,2292728,153919,1645746,9488,...,18.5,-888888888,0.2,92.0,7.8,-888888888,0.0,97.7,2.2,0400000US08]
6,"[""0400000US09""",Connecticut,1442969,1382529,1174225,39129,1312028,125086,944947,13373,...,24.6,-888888888,0.2,89.6,10.2,-888888888,0.2,97.3,2.5,0400000US09]
7,"[""0400000US10""",Delaware,412048,398260,329757,8175,381174,41133,269796,3545,...,22.3,-888888888,0.1,90.1,9.8,-888888888,0.0,97.3,2.7,0400000US10]
8,"[""0400000US11""",District of Columbia,334673,322682,288296,4824,313617,21718,217468,2405,...,28.9,-888888888,0.0,90.5,9.5,-888888888,0.0,97.3,2.7,0400000US11]
9,"[""0400000US12""",Florida,8966402,8707628,7360939,204867,8349941,802786,5935234,57846,...,18.8,-888888888,0.1,91.6,8.3,-888888888,0.0,96.8,3.2,0400000US12]


In [6]:
url = "https://api.census.gov/data/2022/acs/acs5?get=group(B03001)&ucgid=pseudo(0400000US36$8600000)"
zips = pd.read_csv(url)
table = census_vars.rename_columns(zips)
table

,B03001_002E,B03001_003E,B03001_004E,B03001_005E,B03001_006E,B03001_007E,B03001_008E,B03001_009E,B03001_010E,B03001_011E,...,B03001_025E,B03001_026E,B03001_027E,B03001_028E,B03001_029E,B03001_030E,B03001_031E,geography,NAME,ucgid]
0,1044,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,860Z200US12451,ZCTA5 12451,860Z200US12451]
1,3731,70,0,44,0,0,0,0,0,0,...,0,0,22,22,0,0,0,860Z200US13084,ZCTA5 13084,860Z200US13084]
2,313,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,860Z200US12856,ZCTA5 12856,860Z200US12856]
3,915,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,860Z200US12985,ZCTA5 12985,860Z200US12985]
4,2869,9,0,2,0,0,0,0,0,0,...,0,0,6,0,6,0,0,860Z200US13332,ZCTA5 13332,860Z200US13332]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,1350,19,2,17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,860Z200US14478,ZCTA5 14478,860Z200US14478]
1822,4324,454,0,349,0,0,42,0,21,0,...,0,0,63,28,0,0,35,860Z200US14510,ZCTA5 14510,860Z200US14510]
1823,4685,221,53,88,0,71,0,0,0,0,...,0,0,9,3,6,0,0,860Z200US14423,ZCTA5 14423,860Z200US14423]
1824,6917,1688,0,471,104,151,327,0,243,0,...,0,0,151,130,0,0,21,860Z200US10920,ZCTA5 10920,860Z200US10920]
